<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/AGENTIC-MODULE3-MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Ultra-Simple MLP - Minimal Architecture
# Clean implementation with line chart for predictions vs actual
# Now with Huber loss + tail sample weights for [2–4] and [20–24]

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error

class UltraSimpleMLP:
    """
    Ultra-simple MLP with minimal architecture, Huber loss, and tail weights
    """

    def __init__(self, output_dir='/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/'):
        self.output_dir = output_dir
        self.y_scaler = StandardScaler()
        self.x_scaler = StandardScaler()
        self.model = None

        # Create ultra-simple model directory
        self.ultra_dir = f"{output_dir}ultra_simple_mlp/"
        os.makedirs(self.ultra_dir, exist_ok=True)
        print(f"📁 Ultra-Simple MLP directory: {self.ultra_dir}")

    # -------------------------
    # Data I/O
    # -------------------------
    def load_data(self, data_filename, windows_filename):
        """Load data"""
        print("📊 Loading data...")

        data_path = os.path.join(self.output_dir, data_filename)
        windows_path = os.path.join(self.output_dir, windows_filename)

        x = np.load(data_path)
        y = np.load(windows_path)

        print(f"✅ Data loaded: X={x.shape}, y={y.shape}")
        return x, y

    # -------------------------
    # Tail Weights
    # -------------------------
    def make_tail_sample_weights(self, y_unscaled, low=(2, 4), high=(20, 24),
                                 tail_w=3.0, mid_w=1.0, dtype=np.float32):
        """
        Build per-sample weights to up-weight extremes (in ORIGINAL units).
        """
        y = y_unscaled.reshape(-1)
        w = np.full_like(y, fill_value=mid_w, dtype=dtype)
        low_mask  = (y >= low[0])  & (y <= low[1])
        high_mask = (y >= high[0]) & (y <= high[1])
        w[low_mask | high_mask] = tail_w
        return w

    # -------------------------
    # Features
    # -------------------------
    def add_key_interaction(self, x):
        """
        Add the best interaction feature (feat_7 × feat_8) that was found in original analysis
        """
        print(f"🔧 Adding key interaction to {x.shape[1]} features...")

        if x.shape[1] > 8:
            best_interaction = (x[:, 7] * x[:, 8]).reshape(-1, 1)
            x_enhanced = np.hstack([x, best_interaction])
            print(f"   ✅ Added feat_7 × feat_8 interaction")
            print(f"   Features: {x.shape[1]} → {x_enhanced.shape[1]}")
            return x_enhanced
        else:
            print(f"   ⚠️ Not enough features for interaction")
            return x

    def no_feature_selection(self, x, y):
        """
        Use ALL original features - no selection at all
        """
        print(f"🎯 Using ALL original features: {x.shape[1]} features...")
        X_selected = x.copy()
        selected_indices = np.arange(x.shape[1])
        print(f"   Using all {X_selected.shape[1]} original features")
        print(f"   Feature range: 0 to {x.shape[1]-1}")
        return X_selected, selected_indices

    # -------------------------
    # Model
    # -------------------------
    def build_ultra_simple_model(self, input_dim, huber_delta=1.0):
        """
        Ultra-minimal architecture
        """
        print(f"🏗️ Building ULTRA-SIMPLE model for {input_dim} features...")

        first_layer_size = max(32, min(64, input_dim))  # Scale with input size

        model = Sequential([
            Dense(first_layer_size, input_dim=input_dim, activation='relu'),
            Dense(32, activation='relu'),
            Dense(16, activation='relu'),
            Dense(8, activation='relu'),
            Dense(1, activation='linear')
        ])

        optimizer = keras.optimizers.Adam(learning_rate=0.0003,clipnorm=1)

        model.compile(
            loss='mean_squared_error'
            #loss=tf.keras.losses.Huber(delta=huber_delta),  # Huber loss
            optimizer=optimizer,
            metrics=['mean_squared_error']
        )

        print(f"   Architecture: {input_dim} → {first_layer_size} → 32 → 16 → 1")
        print(f"   Ultra-simple model parameters: {model.count_params():,}")
        print(f"   Huber delta: {huber_delta}")
        return model

    def train_ultra_simple_model(self, x_train, x_val, y_train, y_val,
                                 huber_delta=1.0,
                                 sample_weight=None, val_sample_weight=None):
        """
        Quick training with optional sample weights (tail up-weighting)
        """
        print(f"🚀 Training ultra-simple model...")

        self.model = self.build_ultra_simple_model(x_train.shape[1], huber_delta=huber_delta)

        callbacks = [
            EarlyStopping(
                monitor='val_loss',
                patience=30,
                restore_best_weights=True,
                verbose=1
            ),
            ModelCheckpoint(
                f"{self.ultra_dir}ultra_simple_model.weights.h5",
                monitor='val_loss',
                save_best_only=True,
                save_weights_only=True,
                verbose=1
            )
        ]

        history = self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val, val_sample_weight) if val_sample_weight is not None else (x_val, y_val),
            epochs=200,
            batch_size=128,  # slightly smaller than 256 to reduce over-smoothing
            sample_weight=sample_weight,
            callbacks=callbacks,
            verbose=1
        )

        # Save model
        self.model.save(f"{self.ultra_dir}ultra_simple_model.keras")
        print("✅ Ultra-simple model training complete!")

        return history

    # -------------------------
    # Evaluation & Plots
    # -------------------------
    def evaluate_and_plot(self, x_test, y_test, history):
        """Evaluate and create line chart"""
        print("📊 Evaluating ultra-simple model...")

        # Predictions
        y_pred_scaled = self.model.predict(x_test, verbose=0)
        y_pred = self.y_scaler.inverse_transform(y_pred_scaled).flatten()
        y_true = self.y_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()

        # Metrics
        r2 = r2_score(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)

        # Accuracy metrics
        acc_05 = np.mean(np.abs(y_true - y_pred) <= 0.5) * 100
        acc_1 = np.mean(np.abs(y_true - y_pred) <= 1) * 100
        acc_2 = np.mean(np.abs(y_true - y_pred) <= 2) * 100

        print(f"\n📊 ULTRA-SIMPLE MODEL RESULTS:")
        print(f"="*40)
        print(f"R²:           {r2:.6f}")
        print(f"MAE:          {mae:.4f}")
        print(f"RMSE:         {rmse:.4f}")
        print(f"Accuracy ±2:  {acc_2:.1f}%")
        print(f"="*40)

        # Create comprehensive plots
        self.create_result_plots(y_true, y_pred, history, r2, mae)

        return {
            'r2': r2, 'mae': mae, 'rmse': rmse,
            'acc_2': acc_2, 'y_true': y_true, 'y_pred': y_pred
        }

    def create_result_plots(self, y_true, y_pred, history, r2, mae):
        """Create comprehensive result plots including line chart"""

        fig = plt.figure(figsize=(20, 15))

        # 1. Training History
        plt.subplot(3, 3, 1)
        plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
        plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
        plt.title('Training History - Loss', fontsize=14, fontweight='bold')
        plt.xlabel('Epoch')
        plt.ylabel('Loss (Huber)')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 2. MAE History
        plt.subplot(3, 3, 2)
        plt.plot(history.history['mae'], label='Training MAE', linewidth=2)
        plt.plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
        plt.title('Training History - MAE', fontsize=14, fontweight='bold')
        plt.xlabel('Epoch')
        plt.ylabel('Mean Absolute Error')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 3. Predictions Scatter Plot
        plt.subplot(3, 3, 3)
        plt.scatter(y_true, y_pred, alpha=0.6, s=30, edgecolors='white', linewidth=0.5)
        min_val, max_val = min(y_true.min(), y_pred.min()), max(y_true.max(), y_pred.max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect Prediction')
        plt.xlabel('True Values')
        plt.ylabel('Predicted Values')
        plt.title(f'Predictions vs True\nR² = {r2:.4f}', fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 4. LINE CHART - First 100 test samples
        plt.subplot(3, 3, 4)
        n_samples = min(100, len(y_true))
        sample_indices = np.arange(n_samples)
        plt.plot(sample_indices, y_true[:n_samples], '-', linewidth=2, label='True Values', marker='o', markersize=4)
        plt.plot(sample_indices, y_pred[:n_samples], '-', linewidth=2, label='Predictions', marker='s', markersize=4)
        plt.xlabel('Test Sample Index')
        plt.ylabel('Value')
        plt.title(f'Predictions vs True (First {n_samples} samples)', fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # 5. LINE CHART - Last 100 test samples
        plt.subplot(3, 3, 5)
        if len(y_true) > 100:
            start_idx = len(y_true) - 100
            sample_indices = np.arange(start_idx, len(y_true))
            plt.plot(sample_indices, y_true[start_idx:], '-', linewidth=2, label='True Values', marker='o', markersize=4)
            plt.plot(sample_indices, y_pred[start_idx:], '-', linewidth=2, label='Predictions', marker='s', markersize=4)
            plt.xlabel('Test Sample Index')
            plt.ylabel('Value')
            plt.title('Predictions vs True (Last 100 samples)', fontsize=14, fontweight='bold')
            plt.legend()
            plt.grid(True, alpha=0.3)
        else:
            plt.text(0.5, 0.5, 'Not enough samples\nfor separate last 100',
                     ha='center', va='center', transform=plt.gca().transAxes, fontsize=12)
            plt.title('Last 100 Samples (N/A)', fontsize=14, fontweight='bold')

        # 6. Residuals Plot
        plt.subplot(3, 3, 6)
        residuals = y_true - y_pred
        plt.scatter(y_pred, residuals, alpha=0.6, s=30, edgecolors='white', linewidth=0.5)
        plt.axhline(y=0, color='red', linestyle='--', linewidth=2)
        plt.xlabel('Predicted Values')
        plt.ylabel('Residuals (True - Predicted)')
        plt.title('Residuals Plot', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)

        # 7. Error Distribution
        plt.subplot(3, 3, 7)
        plt.hist(residuals, bins=30, alpha=0.7, edgecolor='black')
        plt.xlabel('Residuals')
        plt.ylabel('Frequency')
        plt.title('Residuals Distribution', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)

        # 8. Prediction Error Over Range
        plt.subplot(3, 3, 8)
        abs_errors = np.abs(residuals)
        plt.scatter(y_true, abs_errors, alpha=0.6, s=30, edgecolors='white', linewidth=0.5)
        plt.xlabel('True Values')
        plt.ylabel('Absolute Error')
        plt.title('Prediction Error vs True Value', fontsize=14, fontweight='bold')
        plt.grid(True, alpha=0.3)

        # 9. Model Summary Text
        plt.subplot(3, 3, 9)
        plt.axis('off')
        summary_text = f"""
        ULTRA-SIMPLE MODEL SUMMARY

        Architecture: dynamic → 32 → 16 → 1
        Parameters: {self.model.count_params():,}

        Performance Metrics:
        R² Score: {r2:.6f}
        MAE: {mae:.4f}
        RMSE: {np.sqrt(mean_squared_error(y_true, y_pred)):.4f}

        Accuracy Metrics:
        ±0.5: {np.mean(np.abs(y_true - y_pred) <= 0.5)*100:.1f}%
        ±1.0: {np.mean(np.abs(y_true - y_pred) <= 1)*100:.1f}%
        ±2.0: {np.mean(np.abs(y_true - y_pred) <= 2)*100:.1f}%

        Training Info:
        Epochs: {len(history.history['loss'])}
        Final Train Loss: {history.history['loss'][-1]:.4f}
        Final Val Loss: {history.history['val_loss'][-1]:.4f}
        """
        plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes,
                 fontsize=11, verticalalignment='top', fontfamily='monospace',
                 bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgray', alpha=0.8))

        plt.tight_layout()
        plt.savefig(f"{self.ultra_dir}ultra_simple_comprehensive_results.png",
                    dpi=150, bbox_inches='tight')
        plt.show()

        # Additional detailed line chart for all samples
        self.create_detailed_line_chart(y_true, y_pred)

    def create_detailed_line_chart(self, y_true, y_pred):
        """Create detailed line chart for predictions vs actual"""

        plt.figure(figsize=(16, 10))

        # Plot 1: All or sample of 500
        plt.subplot(2, 1, 1)
        if len(y_true) <= 500:
            sample_indices = np.arange(len(y_true))
            plt.plot(sample_indices, y_true, '-', linewidth=1.5, label='True Values', alpha=0.8)
            plt.plot(sample_indices, y_pred, '-', linewidth=1.5, label='Predictions', alpha=0.8)
            plt.title(f'All Test Samples: Predictions vs True Values (n={len(y_true)})',
                      fontsize=14, fontweight='bold')
        else:
            random_indices = np.sort(np.random.choice(len(y_true), 500, replace=False))
            plt.plot(random_indices, y_true[random_indices], '-', linewidth=1.5,
                     label='True Values', alpha=0.8, marker='o', markersize=2)
            plt.plot(random_indices, y_pred[random_indices], '-', linewidth=1.5,
                     label='Predictions', alpha=0.8, marker='s', markersize=2)
            plt.title(f'Random Sample: Predictions vs True Values (500 of {len(y_true)} samples)',
                      fontsize=14, fontweight='bold')

        plt.xlabel('Sample Index')
        plt.ylabel('Value')
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Plot 2: Zoomed view of interesting region
        plt.subplot(2, 1, 2)
        if len(y_true) > 50:
            window_size = min(50, len(y_true) // 4)
            rolling_var = pd.Series(y_true).rolling(window=window_size).var()
            max_var_idx = int(rolling_var.idxmax())

            start_idx = max(0, max_var_idx - window_size)
            end_idx = min(len(y_true), max_var_idx + window_size)

            zoom_indices = np.arange(start_idx, end_idx)
            plt.plot(zoom_indices, y_true[start_idx:end_idx], '-', linewidth=2,
                     label='True Values', marker='o', markersize=4)
            plt.plot(zoom_indices, y_pred[start_idx:end_idx], '-', linewidth=2,
                     label='Predictions', marker='s', markersize=4)
            plt.title(f'Zoomed View: Most Variable Region (samples {start_idx}-{end_idx})',
                      fontsize=14, fontweight='bold')
        else:
            plt.plot(y_true, '-', linewidth=2, label='True Values', marker='o', markersize=4)
            plt.plot(y_pred, '-', linewidth=2, label='Predictions', marker='s', markersize=4)
            plt.title('Zoomed View: All Samples', fontsize=14, fontweight='bold')

        plt.xlabel('Sample Index')
        plt.ylabel('Value')
        plt.legend()
        plt.grid(True, alpha=0.3)

        plt.tight_layout()
        plt.savefig(f"{self.ultra_dir}detailed_line_chart.png", dpi=150, bbox_inches='tight')
        plt.show()

    # -------------------------
    # Pipeline
    # -------------------------
    def run_ultra_simple_pipeline(self, data_filename, windows_filename,
                                  huber_delta=1.0, tail_weight=3.0):
        """Run the complete ultra-simple pipeline"""
        print("="*60)
        print("⚡ ULTRA-SIMPLE MLP PIPELINE (Huber + Tail Weights)")
        print("="*60)

        # 1. Load data
        x, y = self.load_data(data_filename, windows_filename)

        # 2. Add key interaction feature
        x_enhanced = self.add_key_interaction(x)

        # 3. No feature selection - use all enhanced features
        X_selected, selected_indices = self.no_feature_selection(x_enhanced, y)

        # 3. Split data
        print("📊 Splitting data...")
        x_temp, x_test, y_temp, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)
        x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.25, random_state=42)

        # >>> Build weights from UN-SCALED y <<<
        print("⚖️ Building tail sample weights...")
        train_weights = self.make_tail_sample_weights(y_train, low=(2,4), high=(20,24), tail_w=tail_weight)
        val_weights   = self.make_tail_sample_weights(y_val,   low=(2,4), high=(20,24), tail_w=tail_weight)

        # 4. Scale data
        print("⚖️ Scaling data...")
        x_train_scaled = self.x_scaler.fit_transform(x_train)
        x_val_scaled   = self.x_scaler.transform(x_val)
        x_test_scaled  = self.x_scaler.transform(x_test)

        y_train_scaled = self.y_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
        y_val_scaled   = self.y_scaler.transform(y_val.reshape(-1, 1)).flatten()
        y_test_scaled  = self.y_scaler.transform(y_test.reshape(-1, 1)).flatten()

        print(f"   Train: {x_train_scaled.shape[0]} samples, {x_train_scaled.shape[1]} features")
        print(f"   Val:   {x_val_scaled.shape[0]} samples")
        print(f"   Test:  {x_test_scaled.shape[0]} samples")

        # 5. Train ultra-simple model (with weights)
        history = self.train_ultra_simple_model(
            x_train_scaled, x_val_scaled, y_train_scaled, y_val_scaled,
            huber_delta=huber_delta,
            sample_weight=train_weights,
            val_sample_weight=val_weights
        )

        # 6. Evaluate with detailed plots
        results = self.evaluate_and_plot(x_test_scaled, y_test_scaled, history)

        print("\n" + "="*60)
        print("⚡ ULTRA-SIMPLE MODEL COMPLETE!")
        print("="*60)

        return {
            'model': self.model,
            'results': results,
            'history': history,
            'ultra_dir': self.ultra_dir
        }


# Quick execution function
def run_ultra_simple_mlp(data_filename, windows_filename,
                         output_dir='/content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/',
                         huber_delta=1.0, tail_weight=3.0):
    """
    Run ultra-simple MLP with line charts, Huber loss, and tail weights
    """
    ultra_mlp = UltraSimpleMLP(output_dir)
    return ultra_mlp.run_ultra_simple_pipeline(
        data_filename, windows_filename,
        huber_delta=huber_delta, tail_weight=tail_weight
    )


# Main execution
if __name__ == "__main__":
    print("⚡ Ultra-Simple MLP with Line Charts (Huber + Tail Weights)")

    # Use your data files
    data_file = 'generated-data-OPTIMIZED.npy'
    windows_file = 'generated-data-true-window-OPTIMIZED.npy'

    # You can tune these two knobs:
    HUBER_DELTA = 1.0   # try {0.5, 1.0, 2.0}
    TAIL_WEIGHT = 3.0   # try {2.0, 3.0, 4.0}

    results = run_ultra_simple_mlp(
        data_file, windows_file,
        huber_delta=HUBER_DELTA, tail_weight=TAIL_WEIGHT
    )

    if results:
        r2 = results['results']['r2']
        mae = results['results']['mae']
        acc_2 = results['results']['acc_2']

        print(f"\n⚡ ULTRA-SIMPLE MODEL RESULTS:")
        print(f"="*40)
        print(f"R²:           {r2:.6f}")
        print(f"MAE:          {mae:.4f}")
        print(f"Accuracy ±2:  {acc_2:.1f}%")
        print(f"Parameters:   {results['model'].count_params():,}")
        print(f"="*40)


⚡ Ultra-Simple MLP with Line Charts (Huber + Tail Weights)
📁 Ultra-Simple MLP directory: /content/drive/MyDrive/PHD/2025/TEMP_OUTPUT_METROPM/ultra_simple_mlp/
⚡ ULTRA-SIMPLE MLP PIPELINE (Huber + Tail Weights)
📊 Loading data...
✅ Data loaded: X=(350000, 650), y=(350000,)
🔧 Adding key interaction to 650 features...
   ✅ Added feat_7 × feat_8 interaction
   Features: 650 → 651
🎯 Using ALL original features: 651 features...
   Using all 651 original features
   Feature range: 0 to 650
📊 Splitting data...
⚖️ Building tail sample weights...
⚖️ Scaling data...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')